In [1]:
import lib.datautility as du
import numpy as np
import lib.tf_network as tfnet
import lib.evaluationutility as eu
from lib.tf_network import Network, Cost, Normalization, Optimizer
import tensorflow as tf

c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\anth

In [1]:
import zipfile
with zipfile.ZipFile('resources/resources.zip','r') as zf:
    zf.extractall('resources/')

In [2]:
data, headers = du.read_csv('resources/remnant.csv', 100)
du.print_descriptives(data, headers)

data, headers = du.read_csv('resources/experimental.csv', 100)
du.print_descriptives(data, headers)

-- loading resources/remnant.csv...(100%)   
DESCRIPTIVES     100 Rows, 33 Columns   
0 is_experiment (int):         0 missing (0.0%)     mean=0.25 (SD=0.43)                                                                              
1 user_id (int):               0 missing (0.0%)     mean=38967.20 (SD=16785.77)                                                                      
2 assignment_id (int):         0 missing (0.0%)     mean=423624.56 (SD=328161.88)                                                                    
3 sequence_id (int):           0 missing (0.0%)     mean=11379.34 (SD=22730.82)                                                                      
4 start_time (string):         0 missing (0.0%)     100 unique values                                                                                
5 end_time (string):           43 missing (0.4%)    58 unique values                                                                                 
6 ex_sequence_

In [3]:
seq = dict()

key = [1]
label = [[13], [15], [12]]
cov = [7, 8, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
iden = [0, 2, 3, 6]
sortby = [4]

seq = tfnet.format_sequence_from_file('resources/remnant.csv',key,label,cov,iden,sortby)
print('formatting identifiers...')
seq['key'] = tfnet.fold_by_key(seq['key'], -1, 10)
print('formatting output labels...')
seq['y'] = tfnet.offset_multi_label(seq['y'],2,-1)
print('formatting feature columns...')
seq['x'] = tfnet.sequence_impute_missing(seq['x'])
print('done!')

-- loading resources/remnant.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(0.55%) 

C:\Users\Anthony\Projects\Rebar\Replication\Study_1\lib\tf_network.py:2000: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['x'] = np.array(x)
C:\Users\Anthony\Projects\Rebar\Replication\Study_1\lib\tf_network.py:2002: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['iden'] = np.array(id)


-- formatting sequence...(100%)   


c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


formatting identifiers...
formatting output labels...
formatting feature columns...
done!


In [4]:
seqt = dict()

key = [33, 1]
label = [[13], [15], [12]]
cov = [7, 8, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
iden = [38, 34, 2, 3]
sortby = [4]

seqt = tfnet.format_sequence_from_file('resources/experimental.csv', key, label, cov, iden, sortby)
print('formatting output labels...')
seqt['y'] = tfnet.offset_multi_label(seqt['y'], 2, -1)
seqt['y'] = tfnet.extract_from_multi_label(seqt['y'], [0, 1])
print('formatting feature columns...')
seqt['x'] = tfnet.sequence_impute_missing(seqt['x'])
print('done!')

-- loading resources/experimental.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(100%)   
formatting output labels...
formatting feature columns...
done!


In [5]:
max_epochs = 200
hidden = 50
batch = 64
keep = .5
step = 5e-4
threshold = .001
optimizer = Optimizer.ADAM

n_cov = len(seq['x'][0][0])
seq['y'] = tfnet.extract_from_multi_label(seq['y'], [0, 1])
desc = tfnet.describe_multi_label(seq['y'], True)

exp = np.unique(seqt['key'].reshape((-1, len(key)))[:, 0])

======  Label Description  =============
-- Number of Samples: 130678
-- Number of Label Sets: 2
---- 1: 1 Label
---- 2: 1 Label



In [6]:
tf.reset_default_graph()
tf.set_random_seed(1)
np.random.seed(1)

# build the model
net = Network('study1_model').add_input_layer(n_cov, normalization=Normalization.Z_SCORE)
net.add_lstm_layer(hidden, peepholes=True, activation=tf.nn.leaky_relu)
net.begin_multi_output(cost_methods=[Cost.BINARY_CROSS_ENTROPY,
                                     Cost.RMSE])
net.add_dropout_layer(1, keep=keep, activation=tf.nn.sigmoid)
net.add_dropout_layer(1, keep=keep, activation=tf.identity)
net.end_multi_output()

# set defaults
net.set_default_cost_method(Cost.BINARY_CROSS_ENTROPY)
net.set_optimizer(optimizer)

# train the model and save the trained weights
net.train(x=seq['x'],
          y=seq['y'],
          step=step,
          use_validation=True,
          max_epochs=max_epochs, threshold=threshold, batch=batch)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING - Incomplete validation set information was provided. Using default 70/30% split.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

            Training Network            
-3 layers: 16n -> 50n -> 1n, 1n
-200 epochs
-step size = 0.0005
-batch size = 64
Epoch        Cost    Val Cost   Delta         Time


C:\Users\Anthony\Projects\Rebar\Replication\Study_1\lib\tf_network.py:1047: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([a[i] for a in series_label])]).reshape((len(series_batch), n_timestep, -1)))


Epoch 1     0.9793    0.7230    0.0000       94.5s
Epoch 2     0.9074    0.7193    0.0019       95.0s
Epoch 3     0.9000    0.7170    0.0014       98.9s
Epoch 4     0.8970    0.7148    0.0012      109.4s
Epoch 5     0.8951    0.7147    0.0008      107.4s
Epoch 6     0.8933    0.7136    0.0019      110.8s
Epoch 7     0.8919    0.7137    0.0011      113.0s
Epoch 8     0.8914    0.7128    0.0009      113.7s
Total Time: 842.6s


In [7]:
net.save_model_weights()      # comment to use a previously-saved model
#
# net.build()                 # uncomment to use previously-saved model
# net.restore_model_weights() # uncomment to use previously-saved model

In [8]:
for e in range(len(exp)):
    print('formatting model predictions for experiment {}'.format(exp[e]))
    test = np.argwhere(seqt['key'].reshape((-1, len(key)))[:, 0] == exp[e]).ravel()

    pred = net.predict(x=seqt['x'][test], batch=1024)

    fold_pred = tfnet.flatten_sequence(seqt['y'][test], key=seqt['key'].reshape((-1, len(key)))[test], identifier=seqt['iden'][test])

    for p in range(len(pred)):
        tr_max = 1
        tr_min = 0
        upperbound = 1 if p == 0 else 0.333

        # and apply it to the experimental predictions
        fold_pred = np.append(fold_pred,
                              ((tfnet.flatten_sequence(pred[p]) - tr_min) / (tr_max - tr_min)) * upperbound,
                              axis=1)

    hdr = np.array(headers)[iden]
    hdr = np.append(hdr, np.array(headers)[key])

    hdr = np.append(hdr, ['complete', 'inv_mastery', 'p_complete', 'p_inv_mastery'])
    du.write_csv(fold_pred, 'model_predictions.csv', hdr if e == 0 else None, append=e > 0)
print('done!')

formatting model predictions for experiment 226210


C:\Users\Anthony\Projects\Rebar\Replication\Study_1\lib\tf_network.py:1427: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pred.append(np.array(out_p))
C:\Users\Anthony\Projects\Rebar\Replication\Study_1\lib\tf_network.py:1433: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pred[i] = np.append(pred[i], np.array(out_p))


formatting model predictions for experiment 237447
formatting model predictions for experiment 241501
formatting model predictions for experiment 241622
formatting model predictions for experiment 243393
formatting model predictions for experiment 246482
formatting model predictions for experiment 246627
formatting model predictions for experiment 246647
formatting model predictions for experiment 250476
formatting model predictions for experiment 255116
formatting model predictions for experiment 256017
formatting model predictions for experiment 256027
formatting model predictions for experiment 259379
formatting model predictions for experiment 263015
formatting model predictions for experiment 263052
formatting model predictions for experiment 263057
formatting model predictions for experiment 263109
formatting model predictions for experiment 263115
formatting model predictions for experiment 293151
formatting model predictions for experiment 303899
formatting model predictions fo

In [9]:
net.session.close()